### 중앙 일보 데이터크롤러

```
Arguments : query_word, (period)

return : title_list, category_list, body_list
```

In [9]:
import requests
from bs4 import BeautifulSoup

In [20]:
def body_extractor(link):
    soup = BeautifulSoup(requests.get(link).content, 'html.parser')
    
    category = soup.select_one("meta[property='article:section']")["content"]

    return category, soup.select("#article_body")[0].text.strip().replace("\xa0", "").replace("▲", "").replace("▲", "")

def joongang_crawler(query, period=None):
    
    # data containers 
    titles = []
    links = []
    categories = []
    dates = []
    bodies = []
    
    i = 1 
    nobody = 0
    
    while True:
        print("-" * 30)
        print('{} page is start'.format(i))
        
        url = "https://search.joins.com/TotalNews?page={}&Keyword={}&SortType=New&SearchCategoryType=TotalNews".format(i, query)
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'html.parser')
        
        if len(soup.select(".guide")) != 0:
            print('***** no result! *****')
            break

        for art, da in zip(soup.select(".headline > a"), soup.select(".byline > em:nth-child(2)")):
            
            title = art.text
            if len(title) == 0:
                nobody += 1
                continue
                
            link = art["href"]
            if len(link) == 0:
                nobody += 1
                continue
                    
            date = da.text
            if len(date) == 0:
                date = "unknown_date"
            
            try:
                category, body = body_extractor(link)
                if len(body) == 0:
                    print("no body")
                    nobody += 1
                    continue
                if len(category) == 0:
                    category = 'unknown_category'
            
            except Exception as ex:
                print(ex)
                nobody += 1
                continue
                
            titles.append(title)
            links.append(link)
            categories.append(category) 
            dates.append(date)
            bodies.append(body)
            
        print('{} page is done'.format(i))
        i += 1
        break
    return [titles, links, categories, dates, bodies], nobody

In [21]:
data_dust, nobody_dust = joongang_crawler("미세먼지", period=None)

------------------------------
1 page is start
1 page is done


In [22]:
data_dust

[['[6월 5일 PM2.5] 오후 5시 전국 초미세먼지 현황',
  '에코밸류(EVC) 재단, 글로벌 유저와 함께 제주도에서 열린 시연회 진행',
  '오늘은 환경의 날…창원서 전국 첫 수소 시내버스 운행',
  "'달리는 공기청정기'… 현대車, 신형 수소전기버스 공개",
  '한국암웨이, 미래재단 설립 통해 사회공헌활동 등 사업 추진',
  '[6월 5일 PM2.5] 오전 11시 전국 초미세먼지 현황',
  '[오늘의 날씨 키워드] 폭염주의보·미세먼지·서해안 안개',
  '[날씨] 폭염주의보 확대…내일 전국에 비',
  '[오늘의 날씨] 전남·영남 폭염주의보…오전 서해안 짙은 안개',
  '[6월 5일 PM2.5] 오전 6시 전국 초미세먼지 현황'],
 ['https://news.joins.com/article/23489475',
  'https://news.joins.com/article/23489412',
  'https://news.joins.com/article/23489380',
  'https://news.joins.com/article/23489372',
  'https://news.joins.com/article/23489305',
  'https://news.joins.com/article/23489171',
  'https://news.joins.com/article/23488992',
  'https://news.joins.com/article/23488954',
  'https://news.joins.com/article/23488935',
  'https://news.joins.com/article/23488902'],
 ['사회', '문화', '사회', '경제', '사회', '사회', '사회', '사회', '사회', '사회'],
 ['2019.06.05 17:50',
  '2019.06.05 16:34',
  '2019.06.05 15:51',
  '2019.06.05 15:43',
  '2019.06.05 14:40',
  '20